<a href="https://colab.research.google.com/github/chelynl/sentiment_analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources:<br>
https://github.com/hellotinah/youtube_sentiment_analysis/blob/main/comments_full_analysis.ipynb<br>
https://medium.com/analytics-vidhya/whatsapp-chat-analysis-using-python-88123e63c8ae<br>
https://regex101.com/

In [ ]:
# import packages
import pickle
import re
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from datetime import datetime

# !pip install emoji
# !pip install vaderSentiment
import emoji
from emoji import UNICODE_EMOJI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# !pip install TextBlob
from textblob import TextBlob
from textblob import Word

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Between Messages/between_msgs.xlsx')

In [ ]:
# remove unwanted cols
data = data['DATA']

In [ ]:
data

0                                   Saturday, April 8, 2017
1                   PM 02:33, 💍 Wifey Bear ❤️ : Hiiii boooo
2                         PM 02:33, 💍 Hubby Bear 🐻 : First 
3                    PM 02:39, 💍 Hubby Bear 🐻 : (milk_face)
4                        PM 02:40, 💍 Hubby Bear 🐻 : STICKER
                                ...                        
158438                          Thursday, December 31, 2020
158439       AM 11:02, 💍 Wifey Bear ❤️ : Good morning bear!
158440    AM 11:02, 💍 Wifey Bear ❤️ : Hope you have a fu...
158441    AM 11:02, 💍 Wifey Bear ❤️ : Hehehe and see you...
clean     [Saturday, April 8, 2017, PM 02:33,  Wifey Bea...
Name: DATA, Length: 158443, dtype: object

In [ ]:
# 3153 na values
sum(data.isna())

# drop na values
data.dropna(inplace=True)

In [ ]:
data

0                                   Saturday, April 8, 2017
1                   PM 02:33, 💍 Wifey Bear ❤️ : Hiiii boooo
2                         PM 02:33, 💍 Hubby Bear 🐻 : First 
3                    PM 02:39, 💍 Hubby Bear 🐻 : (milk_face)
4                        PM 02:40, 💍 Hubby Bear 🐻 : STICKER
                                ...                        
158438                          Thursday, December 31, 2020
158439       AM 11:02, 💍 Wifey Bear ❤️ : Good morning bear!
158440    AM 11:02, 💍 Wifey Bear ❤️ : Hope you have a fu...
158441    AM 11:02, 💍 Wifey Bear ❤️ : Hehehe and see you...
clean     [Saturday, April 8, 2017, PM 02:33,  Wifey Bea...
Name: DATA, Length: 155290, dtype: object

In [ ]:
# function for removing emoticons
def remove_emojis(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    return regex_pattern.sub(r'',text)

In [ ]:
# list to hold clean text without emojis
no_emojis_text = []

# iterate each row and remove emojis and append to clean list
for row in data:
  if isinstance(row, str)==True:
    clean_row = remove_emojis(row)
    no_emojis_text.append(clean_row)
  else:
    no_emojis_text.append(row)

# add clean values as new col
removed_emojis = pd.DataFrame(no_emojis_text)

In [ ]:
removed_emojis

,0
0,"Saturday, April 8, 2017"
1,"PM 02:33, Wifey Bear : Hiiii boooo"
2,"PM 02:33, Hubby Bear : First"
3,"PM 02:39, Hubby Bear : (milk_face)"
4,"PM 02:40, Hubby Bear : STICKER"
...,...
155285,"Thursday, December 31, 2020"
155286,"AM 11:02, Wifey Bear : Good morning bear!"
155287,"AM 11:02, Wifey Bear : Hope you have a fun t..."
155288,"AM 11:02, Wifey Bear : Hehehe and see you to..."


In [ ]:
sum(removed_emojis[0].isna())

0

In [ ]:
date = datetime.strptime(removed_emojis.iloc[0,0], '%A, %B %d, %Y').date()


In [ ]:
pattern = re.compile('\w+,\s\w+\s\d*,\s\d+')

In [ ]:
messengers = re.findall(pattern, removed_emojis.iloc[0,0])

['Saturday, April 8, 2017']

In [ ]:
date_pattern = re.compile('\w+,\s\w+\s\d*,\s\d+')

dates_list = []

for row in removed_emojis[0]:
  if isinstance(row, str)==True:
    date = re.findall(date_pattern, row)
    dates_list.append(date)
  else:
    dates_list.append(date)

In [ ]:
date = re.findall(date_pattern, removed_emojis.iloc[1,0])
date[0]

IndexError: ignored

In [ ]:
sum(removed_emojis[0].str.contains(date_pattern).isna())

44